In [41]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import spacy
nlp = spacy.load('en', disable=['parser', 'ner'])
from spacy.lang.en import English
import nltk
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from gensim import corpora
import pickle
import pyLDAvis.gensim
%matplotlib inline

In [2]:
df_rec=pd.read_csv("/Users/Jyo/Documents/GA/summary.csv")

In [3]:
df_desc=df_rec["summary"]

In [4]:
en_stop = set(nltk.corpus.stopwords.words('english'))

In [5]:
parser = English()

def tokenize(text):
    lda_tokens = list()
    tokens = parser(text)
    for token in tokens:
        if token.is_space:
            continue
        elif token.is_punct:
            continue
        elif token.is_stop:
            continue
        elif token.is_digit:
            continue
        elif token.like_email:
            continue    
        elif token.like_url:
            continue  
        elif token.is_alpha:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [6]:

def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    


In [7]:

en_stop.update(
{'experience','client','team','work','people','business','position','world','role','sydney','trains','data','agriculture','victoria',
 'challenge','application','position','apply','communication','confidential', 'discussion','communication','sets','privacy','road','safety','level','transport','nsw',
'opportunity','click','market','bringing','passionate','free','movie','drama','entertainment','love','television','news',
'fun','different','successful','candidate','better','special','end','highly','regard','responsibility','regard','problem','foxtel',
'solving','include','end','non','internal','external','track','record','month', 'recruit','month',"location","join","international","medium","looking","flexible","salary","packaging","benefits","privacy",
"selection", "criteria","confidential","discussion","personal","information","written"," verbal",'skills','require',
'analyst','navigation','planning','qualification','process','word','recruitment','feel','support','empower','idea'
'attention','detail','develop','benefit','asic','vision','wealth','vision','torres','strait','wealth','help','financial','analytical','machine','learning',
'provide','cv','working','hard','online','recruit','talk','flexible','let','idea','range','option','diverse','ensure','sounds',
'like','verbal','write','future','means','fair','strong','selection','train','crew','efficient','chandler','macleod','train', 'crew','plus','supperannuation','marketing',
'fix','term','continuous','improvement','large','complex','solution','analytical','application','life','balance','years','cover',
'letter','cross','equal','employer','visit','website','relevant','discipline','root','cause','decision','day,'
'making','application','close','encourage','prove','ability','responsibility','aboriginal','skill','power','bi','contribute','development','key',
'islander','fast','pace','initial','contract','australian','inclusive','culture','closing','date','best','practices','orientation','cuting',
'edge','high','quality','computer','science','skill','excellent','science','require','meet','requirement','solve','problem','view','description''feel', 'support',
'details','contact','arrangement','motivation','outline','idea','flourish','helping','achieve','individually','large','suitability','empower','idea',
'learn','trust'})


In [8]:
from nltk import ngrams

In [9]:
def prepare_text_for_lda(text):
    texts_out=list()
    tokens = tokenize(text)
    tokens = [token for token in tokens if token not in en_stop]
    tokens=  [get_lemma(token) for token in tokens]
    #n_grams=ngrams(tokens,4)
    #bigrams=ngrams(tokens,2)
    #tokens=list(bigrams)
    bigrm = nltk.bigrams(tokens)
    #print(list(bigrm))
    #trigrm = nltk.trigrams(tokens)
    texts_out= [' '.join(item) for item in list(bigrm)]
    #data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
     
    #print(texts_out)
    
    #return tokens
    return texts_out

In [10]:
text_data=list()
for index,text in df_desc.iteritems():
    text_data.append(prepare_text_for_lda(text))
    

In [11]:

dictionary = corpora.Dictionary(text_data)
#dictionary.filter_extremes(no_below=10,no_above=0.2,keep_n=100000)
corpus = [dictionary.doc2bow(text) for text in text_data]


In [12]:

pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [13]:
import gensim

In [14]:

NUM_TOPICS =6
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')
topics = ldamodel.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.001*"sql server" + 0.000*"advance analytics" + 0.000*"project management" + 0.000*"visualisation tool" + 0.000*"stakeholder management" + 0.000*"predictive model" + 0.000*"r python" + 0.000*"cloud platform" + 0.000*"python r" + 0.000*"cloud base"')
(1, '0.001*"sql server" + 0.000*"professional services" + 0.000*"relational database" + 0.000*"web services" + 0.000*"programming language" + 0.000*"new zealand" + 0.000*"analysis reporting" + 0.000*"project management" + 0.000*"python scala" + 0.000*"innovative solution"')
(2, '0.001*"stakeholder management" + 0.001*"python r" + 0.001*"actionable insight" + 0.001*"customer analytics" + 0.001*"advance analytics" + 0.000*"relational database" + 0.000*"mission australia" + 0.000*"subscription discount" + 0.000*"australia value" + 0.000*"value stream"')
(3, '0.001*"advance analytics" + 0.001*"r python" + 0.001*"python r" + 0.000*"statistical analysis" + 0.000*"senior management" + 0.000*"commitment diversity" + 0.000*"real difference" + 

In [15]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')

lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

C:\Users\Jyo\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [124]:
topf=pd.DataFrame(lda.show_topic(0),columns=['Topic','Freq'])

In [161]:
topf

,Topic,Freq
0,sql server,0.000844
1,advance analytics,0.000485
2,project management,0.000444
3,visualisation tool,0.000426
4,stakeholder management,0.000412
5,predictive model,0.000410
6,r python,0.000393
7,cloud platform,0.000393
8,python r,0.000393
9,cloud base,0.000393


In [27]:
top1=pd.DataFrame(lda.show_topic(1),columns=['Topic','Freq'])

In [93]:
top1

,Topic,Freq
0,sql server,0.000584
1,professional services,0.000475
2,relational database,0.000466
3,web services,0.000434
4,programming language,0.000403
5,new zealand,0.000367
6,analysis reporting,0.000331
7,project management,0.000330
8,python scala,0.000303
9,innovative solution,0.000301


In [95]:
top2=pd.DataFrame(lda.show_topic(2),columns=['Topic','Freq'])
top2

,Topic,Freq
0,stakeholder management,0.000601
1,python r,0.000596
2,actionable insight,0.000578
3,customer analytics,0.000568
4,advance analytics,0.000525
5,relational database,0.000492
6,mission australia,0.000489
7,subscription discount,0.000489
8,australia value,0.000488
9,value stream,0.000488


In [29]:
top3=pd.DataFrame(lda.show_topic(3),columns=['Topic','Freq'])

In [162]:
top3

,Topic,Freq
0,advance analytics,0.000888
1,r python,0.000631
2,python r,0.000505
3,statistical analysis,0.000488
4,senior management,0.000446
5,commitment diversity,0.000444
6,real difference,0.000434
7,analysis reporting,0.000415
8,senior stakeholder,0.000414
9,big technology,0.000384


In [30]:
top4=pd.DataFrame(lda.show_topic(4),columns=['Topic','Freq'])

In [163]:
top4

,Topic,Freq
0,sql server,0.000615
1,artificial intelligence,0.000539
2,r python,0.000478
3,research scientist,0.000463
4,intelligence analytics,0.000463
5,advance analytics,0.000425
6,quantium health,0.000423
7,innovative solution,0.000405
8,stakeholder management,0.000349
9,visualisation tool,0.000349


In [31]:
top5=pd.DataFrame(lda.show_topic(5),columns=['Topic','Freq'])

In [164]:
top5

,Topic,Freq
0,advance analytics,0.000753
1,research scientist,0.000461
2,health service,0.000417
3,hadoop cluster,0.000417
4,federal government,0.000397
5,cloud base,0.000376
6,award winning,0.000376
7,digital transformation,0.000351
8,predictive model,0.000338
9,python r,0.000335
